In [1]:
from time import strftime,localtime

# if you want to reuse the same dup_locs, reuse the old version name
version="zmax_5m"

if version is None:
    version=strftime("%Y%m%d_%H%M", localtime())
print(version)

zmax_5m


In [2]:
SHARED="/data/sharing/QuakeCoRE"

import sys

sys.path.append(f"{SHARED}/Ancillary_tools/CPT_Vsz_Vs30")
sys.path.append(f"{SHARED}/Ancillary_tools")
sys.path.append(f"{SHARED}/qcore")

from collections import Counter

import os
import os.path
from pathlib import Path
import yaml


from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, desc
from sqlalchemy.orm import sessionmaker


from getCPTdata import getCPTdata
from getCPTparam import getCPTparam
from computeVs import Vs_McGann,Vs_Robertson, Vs_Andrus, Vs_Hegazy, Vs_McGann2
from computeVsz import compute_vsz_from_vs
from computeVs30 import vsz_to_vs30
from loc_filter import locs_multiple_records
from qcore import geo


HOME = Path(os.path.expanduser("~"))
out_dir = HOME / f"Data/cpt/outdir/{version}"
plot_dir = out_dir / "validation_plots"

plot_dir.mkdir(parents=True, exist_ok=True)



results = {}



engine = create_engine(f'sqlite:///{SHARED}/Data/nz_cpt.db')
DBSession = sessionmaker(bind=engine)
session = DBSession()
                


def log_error(skipped_fp, cpt_name, error):
    skipped_fp.write(f"{cpt_name} - {error}\n")


def count_digits(arr):
    stringified = str(arr).replace("0", "").replace(".", "")
    return Counter(stringified)


Base = declarative_base()

class CPTLocation(Base):
    __tablename__ = 'cpt_location'
    id = Column(Integer, primary_key=True)
    #    customer_id=Column(Integer, ForeignKey('customers.id'))
    name = Column(String(20), nullable=False)  # 20210427_17
    private = Column(Integer) #true / false
    type = Column(String(5)) # CPT or SCPT
    nztm_x = Column(Float)
    nztm_y = Column(Float)

    def __iter__(self): #overridding this to return tuples of (key,value)
        return iter([('id',self.id),('name',self.name),('nztm_x',self.nztm_x),('nztm_y',self.nztm_y)])


class CPTDepthRecord(Base):
    __tablename__ = 'cpt_depth_record'
    id = Column(Integer, primary_key=True)
    cpt_name = Column(String(20), nullable=False)  # 
    depth = Column(Float) #
    qc = Column(Float) #
    fs = Column(Float)
    u = Column(Float)
    loc_id = Column(Integer, ForeignKey('cpt_location.id'))
    
    def __iter__(self): #overridding this to return tuples of (key,value)
        return iter([('id',self.id),('depth',self.depth),('qc',self.qc),('fs',self.fs),('u',self.u),('loc_id',self.loc_id)])


# not really useful, but presented as an example
def cpt_records(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).all()
    return res

# not really useful, but presented as an example
def max_depth_record(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).order_by(CPTDepthRecord.depth.desc()).first()
    return res

# the following 3 functions are actually used
def cpt_locations():
    return session.query(CPTLocation).all()

def cpt_records_exists(cpt_name):
    res=session.query(CPTDepthRecord).filter(CPTDepthRecord.cpt_name == cpt_name).first()
    return (res is not None)

def get_cpt_data(cpt_name, columnwise=True):
    res=session.query(CPTDepthRecord.depth,CPTDepthRecord.qc,CPTDepthRecord.fs,CPTDepthRecord.u).filter(CPTDepthRecord.cpt_name == cpt_name).all()
    res_array =np.array(res)
    if columnwise: #each column is grouped together
        res_array = res_array.T
    return res_array
    

def cpt_to_vs(correlationName, z, qc, fs, u2):
    (qt, Ic, Qtn, qc1n, qt1n, effStress) = getCPTparam(z, qc, fs, u2)

    if correlationName == 'McGann':
        (z, Vs, Vs_SD) = Vs_McGann(z, qc, fs)

    elif correlationName == 'Andrus':
        (z, Vs, Vs_SD) = Vs_Andrus(Ic, z, qt)

    elif correlationName == 'Robertson':
        (z, Vs, Vs_SD) = Vs_Robertson(z, Ic, Qtn, effStress)

    elif correlationName == 'Hegazy':
        (z, Vs, Vs_SD) = Vs_Hegazy(z, Ic, qc1n, effStress)

    elif correlationName == 'McGann2':
        (z, Vs, Vs_SD) = Vs_McGann2(z, qc, fs)
    
    return z, Vs, Vs_SD
    

# Usage Examples

Check if any record found for a CPT name

In [3]:
print(cpt_records_exists("CPT_108566"))
print(cpt_records_exists("SCPT_140251"))


False
True


In [4]:
get_cpt_data("CPT_108566")


array([], dtype=float64)

Retrieve CPT records for a given CPT

In [5]:
%%time
res=cpt_records('SCPT_140251')
#print(res)
#print(res[0].depth)

CPU times: user 59.1 ms, sys: 3.56 ms, total: 62.6 ms
Wall time: 62.3 ms


However, the following function is preferred

In [6]:
depth, qc, fs, u = get_cpt_data("SCPT_140251")

In [7]:
print(depth)

[ 0.03  0.05  0.07 ... 20.65 20.67 20.69]


With `columnwise=False`, each CPT depth record is grouped together as below. `columnwise=True` by default

In [8]:
cpt_records = get_cpt_data("SCPT_140251", columnwise=False)
print(cpt_records[0])

[ 0.03    0.0105  0.0002 -0.0004]


Retrieves all locations

In [9]:
%%time
locs = cpt_locations()
print(dict(locs[0]))


{'id': 1, 'name': 'CPT_1', 'nztm_x': 1576467.294706431, 'nztm_y': 5181262.382226084}
CPU times: user 961 ms, sys: 76.9 ms, total: 1.04 s
Wall time: 1.04 s


The record at the maximum depth can be retrived by this function. (In practice, this function may not be useful)

In [10]:
%%time
print(dict(max_depth_record('SCPT_140251')))


{'id': 1034, 'depth': 20.69, 'qc': 22.7256, 'fs': 1.0416, 'u': -0.0251, 'loc_id': 39741}
CPU times: user 3.57 ms, sys: 465 µs, total: 4.03 ms
Wall time: 3.28 ms


# Find Duplicate Locations

In [11]:
#dup_locs_dict is a dictionary {loc0:[loc00,loc01],loc1:[loc10,loc11,loc12],loc2:[loc20]...} 
#meaning loc00,loc01 are within 0.1m distance from loc0. It may be too strict.

#computing locs_multiple_records() takes about 5 mins, and I'm skipping here
#if filtering criteria needs to be updated, revise locs_multiple_records() @ loc_filter.py and delete out_dir/dup_locs.yaml

dup_locs_yaml_file = out_dir/"dup_locs.yaml"

if dup_locs_yaml_file.exists():
    with open(out_dir/"dup_locs.yaml", 'r') as f:
        dup_locs_dict = yaml.load(f, Loader=yaml.SafeLoader)
else:        
    dup_locs_dict=locs_multiple_records(locs, stdout=True)
    #let's save this in a yaml file for future use
    with open(out_dir/"dup_locs.yaml","w") as f:
        yaml.safe_dump(dup_locs_dict,f)

    
#flattens dictionary into lists. [loc00,loc01,loc10,loc11,loc12,loc20...]
import functools,operator
dup_locs = functools.reduce(operator.iconcat, list(dup_locs_dict.values()), []) 

#note that loc0,loc1 are not in dup_locs. These locations are to be processed. Duplicates are to be skipped


CPT_10002         172.64549432844217 -43.52471638225848
===1. CPT_6445     172.64549432844217 -43.52471638225848 (dist:0.000000)
CPT_10005         172.63719404833557 -43.52558790573636
===1. CPT_9747     172.63719404833557 -43.52558790573636 (dist:0.000000)
===2. CPT_9780     172.63719404833557 -43.52558790573636 (dist:0.000000)
CPT_10018         172.67399226934313 -43.51533677408961
===1. CPT_17195    172.67399226934313 -43.51533677408961 (dist:0.000000)
CPT_100487        172.6795963849534 -43.55151192551167
===1. CPT_87923    172.6795963849534 -43.55151192551167 (dist:0.000000)
CPT_100496        172.6421776474338 -43.516392914368566
===1. CPT_88242    172.6421776474338 -43.516392914368566 (dist:0.000000)
CPT_1010          172.6512779079427 -43.54089594133181
===1. CPT_502      172.6512779079427 -43.54089594133181 (dist:0.000000)
CPT_101058        172.7037279706582 -43.57404493171889
===1. CPT_103746   172.70372802656306 -43.57404496885039 (dist:0.000006)
CPT_101059        172.7037659

# Main routine

Loops through all `locs`, and for each location, it performs filtering process. When all the filtering criteria is met, it does conversion from CPT to Vs, Vsz and Vs30

In [ ]:
%%time
skipped_fp = open(out_dir / f"skipped_cpts", "w")
output_file = out_dir / f"vs30_results.csv"

log_error(skipped_fp, "<<<<<", f"Beginning loop : {version}")
for row_n, loc in enumerate(locs):
    
    cpt_name = loc.name
    
    if row_n % 1000 == 0: #print every 1000
        print(f"{row_n+1}/{len(locs)} - {cpt_name}")

        
    if not cpt_records_exists(cpt_name):
        log_error(skipped_fp, cpt_name, f"Type 01 : No record found: {cpt_name}")
        continue
    
    z, qc, fs, u2 = get_cpt_data(cpt_name)
     

    # duplicate location
    if cpt_name in dup_locs:
        log_error(skipped_fp, cpt_name, f"Type 02 :Duplicate location")
        continue

    # duplicate depth check
    u, c = np.unique(z, return_counts=True)
    if np.any([c > 1]):
        log_error(skipped_fp, cpt_name, f"Type 03 : Duplicate depth detected - invalid CPT")
        continue

    # Check for invalid negative readings
    if any(fs < -0.2) or any(qc < -0.2) or any(u2 < -0.2):
        log_error(skipped_fp, cpt_name, f"Type 04 : negative value - discarding")
        continue

    # Check for repeated digits
    if any(value > 3 for fs_value in fs for value in count_digits(fs_value).values()):
        log_error(skipped_fp, cpt_name, f"Type 05 : Repeated digit - investigating")
        continue

    max_depth = max(z)
    if max_depth < 5:
        log_error(skipped_fp, cpt_name, f"Type 06 : depth<5: {max_depth}")
        continue
    min_depth = min(z)
    z_span = max_depth - min_depth
    if z_span < 5:
        log_error(skipped_fp, cpt_name, f"Type 07 : depth range <5: {z_span}")
        continue
        
    #All the filtering is complete and this record has survived
    
    (z, Vs, Vs_SD) = cpt_to_vs('McGann', z, qc, fs, u2) #can use "Andrus" "Robertson" "Hegazy" "McGann2" instead
    
    # ChrisDlt's idea: Vs30(z<3m) = min(Vs30(z=3), mean(Vs30(2.5 < z < 3.5) ) ) 

    # vs30 is computed futher down, and is a scalar - so working with Vs instead
    
#     Vs_in_2p5_3p5m = np.mean(Vs[np.where((z>=2.5) & (z<3.5))])
#     try:
#         Vs_at_3m = Vs[np.where(z==3.0)][0][0] #find Vs at z==3. may have no data at exact 3.0m and causes IndexError
#     except IndexError:
#         Vs[np.where(z<3)] = Vs_in_2p5_3p5m #if Vs at z=3m not found, just use mean(Vs(2.5<=z<3.5))
#     else:
#         Vs[np.where(z<3)] = min(Vs_at_3m,Vs_in_2p5_3p5m) #if Vs at z=3m is found, use the minimum of two found above                                           
        
    
    Vsz, max_depth = compute_vsz_from_vs(Vs, z)
    
    vs30 = vsz_to_vs30(Vsz, z)

    vs30_result = {}
    vs30_result["NZTM_X"] = loc.nztm_x
    vs30_result["NZTM_Y"] = loc.nztm_y
    vs30_result["Vsz"] = Vsz
    vs30_result["Vs30"] = vs30
    vs30_result["Zmax"] = max_depth
    vs30_result["Zmin"] = min_depth
    vs30_result["Zspan"] = z_span
    results[cpt_name] = vs30_result
    if row_n < 10:
        fig, ax = plt.subplots()
        ax.plot(fs, z)
        ax.invert_yaxis()
        ax.set_ylabel("Depth")
        ax.set_xlabel("fs")
        ax.grid()
        fig.savefig(plot_dir / f"{cpt_name}_fs.png")
        plt.close(fig)

        fig, ax = plt.subplots()
        lowerVs = np.exp(np.log(Vs) - Vs_SD)
        upperVs = np.exp(np.log(Vs) + Vs_SD)
        ax.plot(Vs, z, "red")
        ax.plot(lowerVs, z, "r--", linewidth=0.5)
        ax.plot(upperVs, z, "r--", linewidth=0.5)
        ax.grid()
        ax.invert_yaxis()
        ax.set_ylabel("Depth")
        ax.set_xlabel("Vs (m/s)")
        ax.set_xlim(0, 600)
        fig.savefig(plot_dir / f"{cpt_name}_Vs.png")
        plt.close(fig)

log_error(skipped_fp, ">>>>>", f"Ending loop : {version}")

1/49321 - CPT_1


/data/sharing/QuakeCoRE/Ancillary_tools/CPT_Vsz_Vs30/calculations.py:22: RuntimeWarning: invalid value encountered in true_divide
  tn = dn / vn
/opt/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: divide by zero encountered in log
/opt/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log
/data/sharing/QuakeCoRE/Ancillary_tools/CPT_Vsz_Vs30/calculations.py:22: RuntimeWarning: invalid value encountered in true_divide
  tn = dn / vn
/opt/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: divide by zero encountered in log
/opt/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log
/data/sharing/QuakeCoRE/Ancillary_tools/CPT_Vsz_Vs30/calculations.py:22: RuntimeWarning: invalid value encountered in true_divide
  tn = dn / vn
/opt/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: divi

1001/49321 - CPT_103928
2001/49321 - CPT_106698
3001/49321 - CPT_11089
4001/49321 - CPT_114078
5001/49321 - CPT_117087
6001/49321 - CPT_12186
7001/49321 - CPT_125211
8001/49321 - CPT_128596
9001/49321 - CPT_132681
10001/49321 - CPT_137341
11001/49321 - CPT_141680
12001/49321 - CPT_145879
13001/49321 - CPT_16553
14001/49321 - CPT_19038
15001/49321 - CPT_2146
16001/49321 - CPT_24500
17001/49321 - CPT_26055
18001/49321 - CPT_2792
19001/49321 - CPT_30110
20001/49321 - CPT_32698
21001/49321 - CPT_34626
22001/49321 - CPT_36850
23001/49321 - CPT_38626
24001/49321 - CPT_417
25001/49321 - CPT_45201
26001/49321 - CPT_48331


In [ ]:
print(len(z))
print(len(Vs))

print(np.where(z<2.5)) # indices
print(z[np.where((z>=2.5) & (z<=3.0))]) # z between 2.5 and 3.0
print(Vs[np.where((z>=2.5) & (z<=3.0))]) # Vs for z between 2.5 and 3.0
print(np.mean(Vs[np.where((z>=2.5) & (z<=3.0))]))
print(Vs[np.where(z==3.0)])

# Save Vs30 Estimates

In [ ]:
result_df = pd.DataFrame.from_dict(results, orient="index")
result_df.to_csv(output_file)

ll = geo.wgs_nztm2000x(result_df[["NZTM_X", "NZTM_Y"]])
result_df["lon"] = ll[:, 0]
result_df["lat"] = ll[:, 1]

result_df.plot.scatter("Vs30", "Vsz")
fig = plt.gcf()
fig.savefig(plot_dir / "vs30_vsz_scatter.png")

In [ ]:
result_df